In [ ]:
# cell 1: imports
import os
import uuid
from moviepy.editor import VideoFileClip
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, Audio, Video, clear_output

# cell 2: setup temp dir + load pipeline
TEMP_DIR = "temp"
os.makedirs(TEMP_DIR, exist_ok=True)

classifier = pipeline("audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")


In [ ]:
# cell 3: widgets
upload = widgets.FileUpload(accept=".mp4,.mov,.avi,.mkv,.wav,.mp3", multiple=False)
out = widgets.Output()
display(widgets.HTML("<h2>🎧 Audio Sentiment Analyzer</h2>"))
display(upload, out)


In [ ]:
# cell 4: handle file upload + inference
def handle_upload(change):
    out.clear_output()
    uploaded_file = upload.value
    if not uploaded_file:
        return

    for fname, file_info in uploaded_file.items():
        file_ext = fname.split(".")[-1].lower()
        is_video = file_ext in ["mp4", "mov", "avi", "mkv"]
        unique_id = str(uuid.uuid4())
        temp_input_path = os.path.join(TEMP_DIR, f"{unique_id}.{file_ext}")
        temp_audio_path = os.path.join(TEMP_DIR, f"{unique_id}.wav")

        with open(temp_input_path, "wb") as f:
            f.write(file_info['content'])

        try:
            with out:
                clear_output()
                if is_video:
                    display(Video(temp_input_path, embed=True))
                    clip = VideoFileClip(temp_input_path)
                    audio = clip.audio
                    audio.write_audiofile(temp_audio_path, logger=None)
                    clip.close()
                    if audio:
                        audio.close()
                else:
                    display(Audio(temp_input_path, autoplay=False))
                    temp_audio_path = temp_input_path

                print("Analyzing sentiment...")
                prediction = classifier(temp_audio_path)
                top_result = prediction[0]
                display(widgets.HTML(
                    f"<b>🎧 Sentiment:</b> <span style='color:green'>{top_result['label']}</span> "
                    f"(Confidence: {top_result['score']:.2f})"
                ))

        except Exception as e:
            with out:
                print(f"Error: {e}")

        finally:
            try:
                os.remove(temp_input_path)
                if os.path.exists(temp_audio_path) and is_video:
                    os.remove(temp_audio_path)
            except:
                pass

upload.observe(handle_upload, names='value')
